In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
# set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# function to evaluate loss
def evaluate_loss(X_test_scaled, y_test_scaled, model):
    loss = model.evaluate(X_test_scaled, y_test_scaled, verbose=2)
    print(f'Mean Squared Error on Test Set: {loss}')

In [ ]:
# Load the train dataset
train_dataset = pd.read_excel('/home/framework/coding_python/wqi_ann/src/data_input/train_dataset.xlsx') # replace with the path to your excel file

In [ ]:
# show the first  10 values of the training dataset
train_dataset.head(11)

In [ ]:
# separate features (11 variables) and targer (WQI)
x = train_dataset[['ph', 'Total  Hardness', 'Total Alkalinity', 'Cl', 'NO3', 'SO4', 'F', 'TDS', 'Fe', 'As (ppb)', 'Pb (ppb)']]
y = train_dataset[['WQI']]

In [ ]:
# normalize the features and target using MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)


In [ ]:
# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [ ]:
# build a neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#train the model
model.fit(X_train, y_train, epochs=100, batch_size=8, verbose=2, validation_data=(X_test, y_test))

In [ ]:
# load the test dataset (11 variables without WQI)
test_dataset = pd.read_exel('/home/framework/coding_python/wqi_ann/src/data_input/test_dataset.xlsx')

# extract the 11 variables
X_test_data = test_dataset[['pH', 'Total Hardness', 'Total Alkalinity', 'Cl', 'NO3', 'SO4', 'F', 'TDS', 'Fe', 'As (ppb)', 'Pb (ppb)']]

In [ ]:
# normalize the test data using the same scaler fited to the training data
X_test_data_scaled = scaler_X.transform(X_test_data)

# Predict WQI values using the trained model
predicted_wqi_scaled = model.predict(X_test_data_scaled)

#print scaled prediction using the trained model
print("Scaled prediction:" , predicted_wqi_scaled)

# Inverse transform to get the original WQI values
predicted_wqi = scaler_y.inverse_transform(predicted_wqi_scaled)

#print unscaled prediction for debugging
print("Unscaled predictions:",predicted_wqi)

In [ ]:
# Evaluate the loss of the model on the test set
evaluate_loss(X_test, y_test, model)

In [ ]:
# Add the predicted WQI values to the test dataset
test_dataset['WQI'] = predicted_wqi

# Save the updated test dataset to a new Excel file
output_excel_path = '/home/framework/coding_python/wqi_ann/src/data_output/validation_dataset_output.xlsx'
test_dataset.to_excel(output_excel_path, index=False)

print("WQI values have been added and saved to the output Excel file.")